Ceci n'est pas un commentaire


# ***Google Captcha image recognition***
**A Deep Learning Project using TensorFlow**

*by Emma Begard, Augustin Bouveau, Bagin Jobert-Rollin, Hugues Boisdon*

## I Data **Fetching**

Our source Dataset can be found at : *https://www.kaggle.com/datasets/mikhailma/test-dataset*

*credits : **Mike Mazurov***

### I.1 **Dowloading** Data Files from ***KaggleHub*** Source

In [146]:
import kagglehub

DATA_FOLDER_PATH_IF_CACHED = kagglehub.dataset_download("mikhailma/test-dataset")
print("Path to dataset files in cache:", DATA_FOLDER_PATH_IF_CACHED)

Path to dataset files in cache: C:\Users\Development\.cache\kagglehub\datasets\mikhailma\test-dataset\versions\1


The data files are downloaded in the Users **cache** by default.
If the data source folder is moved, ***please update*** the following *path variable*

In [147]:
DATA_FOLDER_PATH_IF_MOVED = "" # Data folder path if data was moved since download

### I.2 **Loading** Data

#### I.2.A *Functions*

In [148]:
import os
import pandas as pd
from PIL import Image


def loadImagesFromDirectory(directoryPath:str) -> pd.DataFrame:
    data: dict[str:list] = {"Image": [], "Label": []}
    
    for root, directories, files in os.walk(directoryPath):
        for f in files:
            subFolderName = f.split(" ")[0]
            if subFolderName == "Cross"  : subFolderName = "Crosswalk"
            if subFolderName == "Tlight" : subFolderName = "Traffic Light"

            data["Image"].append(Image.open(directoryPath+ "/" + subFolderName + "/"+ f).convert("RGB"))
            data["Label"].append(subFolderName)

    return pd.DataFrame(data)
 


#### I.2.B Loading **raw** image data in a *Dataframe*

In [149]:
imgFolderPath = DATA_FOLDER_PATH_IF_CACHED+"/Google_Recaptcha_V2_Images_Dataset/images" if DATA_FOLDER_PATH_IF_MOVED == "" else DATA_FOLDER_PATH_IF_MOVED+"/Google_Recaptcha_V2_Images_Dataset/images"

rawImagesDf = loadImagesFromDirectory(imgFolderPath)
rawImagesDf.head(5)

,Image,Label
0,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle
1,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle
2,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle
3,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle
4,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle


For now we have loaded data in a *pandas Dataframe*, with images as *Image* (class from PIL library).

## II Data **Preprocessing**

### II.1 **Cleaning** Data

#### II.1.A *Images Size* Normalization 

##### II.1.A.a Functions

In [150]:
def checkImagesSizeInDataframe(df:pd.DataFrame) -> None:
    sizeCounts = {}
    for image in df["Image"]:
        if not(image.size in sizeCounts):
            sizeCounts[image.size] = 1
        else :
            sizeCounts[image.size] += 1
    print(f"Sizes found in Dataframe : {sizeCounts}")

In [151]:
from PIL import ImageOps
from copy import deepcopy

def normalizeImageSizes(df:pd.DataFrame, imgTargetSize:tuple[int,int] = (120, 120)) -> pd.DataFrame:
    dataframe = deepcopy(df)
    numOfCropped = 0; numOfExpandeds = 0
    for index, row in dataframe.iterrows():
        width  = row["Image"].width
        height = row["Image"].height
        
        noPaddings  = [0   , 0  ,     0,      0]
        noCroppings = [0   , 0  , width, height]
                    #  Left, Top, Right, Bottom
        
        paddings  = deepcopy(noPaddings)
        croppings = deepcopy(noCroppings)
        
        if width < imgTargetSize[0]:
            pixelsToAdd = imgTargetSize[0] - width
            paddings[0] = pixelsToAdd  // 2
            paddings[2] = (pixelsToAdd - pixelsToAdd  // 2)
        elif width > imgTargetSize[0]:
            pixelsToCrop = width - imgTargetSize[0]
            croppings[0] = pixelsToCrop // 2
            croppings[2] = width - (pixelsToCrop-pixelsToCrop // 2)
        
        if height < imgTargetSize[1]:
            pixelsToAdd = imgTargetSize[1] - height
            paddings[1] = pixelsToAdd  // 2
            paddings[3] = (pixelsToAdd - pixelsToAdd  // 2)
        elif height > imgTargetSize[1]:
            pixelsToCrop = height - imgTargetSize[1]
            croppings[1] = pixelsToCrop // 2
            croppings[3] = height - (pixelsToCrop-pixelsToCrop // 2)
        
        if paddings != noPaddings:
            row["Image"] = ImageOps.expand(row["Image"], border=tuple(paddings), fill='black')
            numOfExpandeds += 1
        if croppings != noCroppings:
            row["Image"] = ImageOps.crop(row["Image"], border=tuple(paddings))
            numOfCropped += 1
    print(f"{numOfCropped} images were cropped to {imgTargetSize}!")
    print(f"{numOfExpandeds} images were expanded to {imgTargetSize}!")
    return dataframe


##### II.1.A.b Normalizing process

In [152]:
# Initial Check
checkImagesSizeInDataframe(rawImagesDf)

normalizedSizeImagesDf = normalizeImageSizes(rawImagesDf)

# Final Check
checkImagesSizeInDataframe(normalizedSizeImagesDf)

Sizes found in Dataframe : {(120, 120): 10705, (100, 100): 1025}
0 images were cropped to (120, 120)!
1025 images were expanded to (120, 120)!
Sizes found in Dataframe : {(120, 120): 11730}


We are now assured to work only with images of size *(120px per 120px)*.

#### II.1.B Transforming Images to *pixel value Arrays*

In [153]:
from copy import deepcopy
import numpy as np

def imagesToPixelArrays(df:pd.DataFrame) -> pd.DataFrame:
    dataframe = deepcopy(df)
    for id, row in dataframe.iterrows():
        row["Image"] = np.array(row["Image"]).astype(np.float32)
    return dataframe

In [154]:
pixelArraysDf = imagesToPixelArrays(normalizedSizeImagesDf)

pixelArraysDf.head(5)

,Image,Label
0,"[[[117.0, 114.0, 104.0], [114.0, 110.0, 101.0]...",Bicycle
1,"[[[54.0, 75.0, 67.0], [58.0, 76.0, 69.0], [66....",Bicycle
2,"[[[38.0, 33.0, 27.0], [37.0, 34.0, 28.0], [37....",Bicycle
3,"[[[49.0, 49.0, 51.0], [55.0, 56.0, 58.0], [72....",Bicycle
4,"[[[135.0, 128.0, 125.0], [143.0, 130.0, 129.0]...",Bicycle


We can now pursue the preprocessing with a more readable and usable data format.

#### II.1.C *Pixel values* Normalization

In [155]:
def normalizePixelValues(df:pd.DataFrame) -> pd.DataFrame:
    dataframe = deepcopy(df)
    for id, row in dataframe.iterrows():
        row["Image"] /= 255
    return dataframe

In [156]:
normalizedPixelArraysDf = normalizePixelValues(pixelArraysDf)

normalizedPixelArraysDf.head(5)

,Image,Label
0,"[[[0.45882353, 0.44705883, 0.40784314], [0.447...",Bicycle
1,"[[[0.21176471, 0.29411766, 0.2627451], [0.2274...",Bicycle
2,"[[[0.14901961, 0.12941177, 0.105882354], [0.14...",Bicycle
3,"[[[0.19215687, 0.19215687, 0.2], [0.21568628, ...",Bicycle
4,"[[[0.5294118, 0.5019608, 0.49019608], [0.56078...",Bicycle


Pixel values are now normalized from **0 to 1** instead of **0 to 255**.

### II.2 Preparing Data for Tensorflow use

In [157]:
import tensorflow as tf

features = np.stack(normalizedPixelArraysDf["Image"].values)
labels   = np.array(normalizedPixelArraysDf["Label"].values)

### II.3 *Splitting* Data in 2 Datasets for Training and Validation.

In [159]:
from sklearn.model_selection import train_test_split

batch_size = 32
img_height = 120
img_width = 120

features_train, features_val, labels_train, labels_val = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)

train_ds = tf.data.Dataset.from_tensor_slices((features_train, labels_train))
val_ds   = tf.data.Dataset.from_tensor_slices((features_val,     labels_val))

## III **Designing** the *Model*